# Final Project Submission

Student name: Bert Tong
Student pace: self paced
Scheduled project review date/time: 
Instructor name: Eli Thomas
Blog post URL:

# Project Goals:

Overall:  Create a record of each unique team in the "database.sqlite" dataset including:
- the name of the team
- total goals scored during 2011 season
- total number of wins earned in 2011 season
- visualization of the w/l for 2011
- win % in rain for 2011 season

Project Plan:
Phase 1:  Complete our Dataset
1. Explore and clean(if necessary) the dataset, identify how to coherently synchronize our data.

2. Update match dataset with "rain" requires additional gathering of data.
    a. Location of home stadium - Use web scraping as necessary 
    b. Day of game (provided) in combination with GPS location applied to Darksky weather API
    c. Using PrecipProbability to determine if there was rain.

Phase 2:  Creating our summary statistics, Creating Visualizations

Phase 3: Upload records to MongoDB, verify entries.

# Phase 1 - Complete Dataset

In [1]:
# getting to know the data provided as database.sqlite, setting up our cursor
import re
import requests
import sqlite3
import pandas as pd
import json
import datetime
import time
url = 'database.sqlite'
conn = sqlite3.connect(url)
c = conn.cursor()

In [2]:
# To begin to answer our questions, we need to first understand how our data is organized in the database file.

# borrowed this from a stackoverflow answer @ 
# https://stackoverflow.com/questions/11996394/is-there-a-way-to-get-a-schema-of-a-database-from-within-python
def printSchema(connection):
    for (tableName,) in connection.execute(
        """
        SELECT NAME from SQLITE_MASTER where TYPE='table' order by NAME;
        """
    ):
        print("{}:".format(tableName))
        for (
            columnID, columnName, columnType,
            columnNotNull, columnDefault, columnPK,
        ) in connection.execute("pragma table_info('{}');".format(tableName)):
            print("  {id}: {name}({type}){null}{default}{pk}".format(
                id=columnID,
                name=columnName,
                type=columnType,
                null=" not null" if columnNotNull else "",
                default=" [{}]".format(columnDefault) if columnDefault else "",
                pk=" *{}".format(columnPK) if columnPK else "",
            ))

In [3]:
printSchema(c)

Matches:
  0: Match_ID(INTEGER) not null *1
  1: Div(TEXT)
  2: Season(INT)
  3: Date(TEXT)
  4: HomeTeam(TEXT)
  5: AwayTeam(TEXT)
  6: FTHG(INT)
  7: FTAG(INT)
  8: FTR(TEXT)


# 1.1 Exploring our Data

In [4]:
# Let's run a quick SQL query to explore what actually in table within our db.
c.execute("""SELECT NAME from SQLITE_MASTER where TYPE='table' order by NAME;""")
df = pd.DataFrame(c.fetchall())
df.columns = [x[0] for x in c.description]
df.head()

,name
0,Matches
1,Teams
2,Teams_in_Matches
3,Unique_Teams
4,sqlite_sequence


In [5]:
# Exploring each of our tables
c.execute("""SELECT * FROM matches
    WHERE Season == 2011;""")
matches = pd.DataFrame(c.fetchall())
matches.columns = [x[0] for x in c.description]
matches.head()

,Match_ID,Div,Season,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR
0,1092,D1,2011,2012-03-31,Nurnberg,Bayern Munich,0,1,A
1,1093,D1,2011,2011-12-11,Stuttgart,Bayern Munich,1,2,A
2,1094,D1,2011,2011-08-13,Wolfsburg,Bayern Munich,0,1,A
3,1095,D1,2011,2011-11-27,Mainz,Bayern Munich,3,2,H
4,1096,D1,2011,2012-02-18,Freiburg,Bayern Munich,0,0,D


In [6]:
c.execute("""SELECT * FROM Teams;""")
teams = pd.DataFrame(c.fetchall())
teams.columns = [x[0] for x in c.description]
print(len(teams))
teams.head()

468


,Season,TeamName,KaderHome,AvgAgeHome,ForeignPlayersHome,OverallMarketValueHome,AvgMarketValueHome,StadiumCapacity
0,2017,Bayern Munich,27,26,15,597950000,22150000,75000
1,2017,Dortmund,33,25,18,416730000,12630000,81359
2,2017,Leverkusen,31,24,15,222600000,7180000,30210
3,2017,RB Leipzig,30,23,15,180130000,6000000,42959
4,2017,Schalke 04,29,24,17,179550000,6190000,62271


In [7]:
c.execute("""SELECT * FROM Teams_in_Matches;""")
tim = pd.DataFrame(c.fetchall())
tim.columns = [x[0] for x in c.description]
tim.head()

,Match_ID,Unique_Team_ID
0,1,26
1,1,46
2,2,26
3,2,42
4,3,26


In [8]:
c.execute("""SELECT * FROM Unique_Teams;""")
unique = pd.DataFrame(c.fetchall())
unique.columns = [x[0] for x in c.description]
print(len(unique))
unique.head()

128


,TeamName,Unique_Team_ID
0,Bayern Munich,1
1,Dortmund,2
2,Leverkusen,3
3,RB Leipzig,4
4,Schalke 04,5


In [9]:
c.execute("""SELECT * FROM sqlite_sequence;""")
sequence = pd.DataFrame(c.fetchall())
sequence.columns = [x[0] for x in c.description]
sequence.head()

,name,seq
0,Matches,46774
1,Unique_Teams,128


In [10]:
# Let's see how many total matches each team played
c.execute("""SELECT t.Unique_Team_ID, t.TeamName, COUNT(m.HomeTeam) as totalMatches
        FROM Unique_Teams t
        JOIN matches m on t.TeamName = m.HomeTeam
        GROUP BY t.Unique_Team_ID
        ORDER BY totalMatches DESC""")
df = pd.DataFrame(c.fetchall())
df.columns = [x[0] for x in c.description]
df.head()

,Unique_Team_ID,TeamName,totalMatches
0,81,Arsenal,460
1,83,Chelsea,460
2,84,Liverpool,460
3,92,Tottenham,460
4,93,Everton,460


In [11]:
# and who had the most wins?  
c.execute("""SELECT t.Unique_Team_ID, t.TeamName, COUNT(m.FTR) as wins
        FROM Unique_Teams t
        JOIN matches m on t.TeamName = m.HomeTeam
        WHERE m.FTR = "H"
        GROUP BY t.Unique_Team_ID
        ORDER BY wins DESC;""")
df = pd.DataFrame(c.fetchall())
df.columns = [x[0] for x in c.description]
df.head()

,Unique_Team_ID,TeamName,wins
0,99,Man United,334
1,1,Bayern Munich,323
2,81,Arsenal,300
3,83,Chelsea,293
4,84,Liverpool,278


# Summary of Data Exploration
It looks like answering Questions 1 - 4 can come from combining a few of these data tables with straightforward SQL 
queries.   Lets get back to that after we finish compiling our data.

# 1.2 - Determining Rain requires much more effort.

to do this, we will be creating a function get_weather() which will access the darksky api to determine "rain"
Our get request will follow this format.

https://api.darksky.net/forecast/{URL}/{Lat},{Long},{date}?query

so we need to be able to provide the following:
- latitude (float)
- longitude (float)
- date (int - UNIX timestamp)

# Finding GPS coordinates.

gitgub user jokecamp has an awesome .csv file with all the gps coordinates of football stadiums 
https://github.com/jokecamp/FootballData/blob/master/other/stadiums-with-GPS-coordinates.csv

After verifying a few stadiums with Google Maps, I have no reason to believe the data to be misleading.
However this data required us adding some of the German stadiums to match our entire dataset.  As result
stadiums.csv file we use is a modified version of jokecamp's GPS efforts.

In [12]:
# reading in our stadium data

stadiums = pd.read_csv("stadiums.csv")

# verifying that our modifications format appropriately.
stadiums.tail()

,Team,FDCOUK,City,Stadium,Capacity,Latitude,Longitude,Country
144,Munich 1860,Munich 1860,Munich,Grünwalder Stadion,21272,48.110833,11.574444,Germany
145,Dresden,Dresden,Dresden,Rudolf-Harbig-Stadion,32066,51.040454,13.749730,Germany
146,Duisburg,Duisburg,Duisburg,MSV-Arena,31514,51.408951,6.777357,Germany
147,Hansa Rostock,Hansa Rostock,Rostock,Ostseestadion,29000,54.084950,12.095189,Germany
148,Erzgebirge Aue,Erzgebirge Aue,Erzgebirge,Erzgebirgsstadion,9390,50.597755,12.711264,Germany


In [13]:
# Merging our stadium data with our match data gives us every datapoint we need before we run the method to determine "rain"
df = pd.merge(matches, stadiums, left_on = "HomeTeam", right_on = "FDCOUK", how="left")
df.head()

,Match_ID,Div,Season,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR,Team,FDCOUK,City,Stadium,Capacity,Latitude,Longitude,Country
0,1092,D1,2011,2012-03-31,Nurnberg,Bayern Munich,0,1,A,1. FC Nürnberg,Nurnberg,Nuremberg,Grundig-Stadion,50000,49.426111,11.125833,Germany
1,1093,D1,2011,2011-12-11,Stuttgart,Bayern Munich,1,2,A,VfB Stuttgart,Stuttgart,Stuttgart,Mercedes-Benz Arena,60441,48.792269,9.232031,Germany
2,1094,D1,2011,2011-08-13,Wolfsburg,Bayern Munich,0,1,A,VfL Wolfsburg,Wolfsburg,Wolfsburg,Volkswagen Arena,30000,52.431944,10.803889,Germany
3,1095,D1,2011,2011-11-27,Mainz,Bayern Munich,3,2,H,1. FSV Mainz 05,Mainz,Mainz,Coface Arena,34000,49.984167,8.224167,Germany
4,1096,D1,2011,2012-02-18,Freiburg,Bayern Munich,0,0,D,SC Freiburg,Freiburg,Freiburg,MAGE SOLAR Stadion,24000,47.988889,7.893056,Germany


In [14]:
# validate that our data is complete and all NaN's have been addressed.
df.FDCOUK.isna().sum()

0

# Getting the weather - Initial Setup
to do this, we will be creating a function WeatherGetter which will take in 3 variables to determine the weather
- latitude (float)
- longitude (float)
- date (int - UNIX timestamp)

first, we need to take our current date format and convert it into a Unix timestamp.  The other two we can keep as our data has it. (we used https://www.tutorialspoint.com/How-to-convert-Python-date-to-Unix-timestamp as reference for this conversion)

In [15]:
# working with a single sample to convert a date with format YYYY-MM-DD to a unix timestamp
import datetime
import time
d = "2012-02-18"
year, month, day = map(int, d.split('-'))
date1 = datetime.date(year, month, day)
unixtime = time.mktime(date1.timetuple())
print(unixtime)

1329544800.0


In [16]:
# create a function to speed up the process

def convert_date(d):
    year, month, day = map(int, d.split('-'))
    date1 = datetime.date(year, month, day)
    unixtime = int(time.mktime(date1.timetuple()))
    return unixtime

In [17]:
# once created we iterate over our df and add into our dataframe.  
# in hindsight, I learned about the apply() after doing this.
ut = []
for d in df.Date:
    ut.append(convert_date(d))
df["unixtime"] = ut
df.head()

,Match_ID,Div,Season,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR,Team,FDCOUK,City,Stadium,Capacity,Latitude,Longitude,Country,unixtime
0,1092,D1,2011,2012-03-31,Nurnberg,Bayern Munich,0,1,A,1. FC Nürnberg,Nurnberg,Nuremberg,Grundig-Stadion,50000,49.426111,11.125833,Germany,1333170000
1,1093,D1,2011,2011-12-11,Stuttgart,Bayern Munich,1,2,A,VfB Stuttgart,Stuttgart,Stuttgart,Mercedes-Benz Arena,60441,48.792269,9.232031,Germany,1323583200
2,1094,D1,2011,2011-08-13,Wolfsburg,Bayern Munich,0,1,A,VfL Wolfsburg,Wolfsburg,Wolfsburg,Volkswagen Arena,30000,52.431944,10.803889,Germany,1313211600
3,1095,D1,2011,2011-11-27,Mainz,Bayern Munich,3,2,H,1. FSV Mainz 05,Mainz,Mainz,Coface Arena,34000,49.984167,8.224167,Germany,1322373600
4,1096,D1,2011,2012-02-18,Freiburg,Bayern Munich,0,0,D,SC Freiburg,Freiburg,Freiburg,MAGE SOLAR Stadion,24000,47.988889,7.893056,Germany,1329544800


# Getting the Weather - Darksky API
Now that we have the variables we need to get our DarkSky response, lets figure out how we need to fit it all in together.
1. get API key and store it as variable
2. Build URL for our get
3. Understand and interpret the data from our json response
4. Identify the datapoint(s) we need and how to access it

In [18]:
# getting our api key

def get_keys(path):
    with open(path) as f:
        return json.load(f)

keys = get_keys("dark_sky_api.json")
api_key = keys['api_key']

In [19]:
# building our function

# notes from DarkSky faq page - https://darksky.net/dev/docs/faq#faq-api-features
# Q: Why isn't precipProbability always 0 or 1 for historical responses?
# A: We don't have perfect historical data—weather stations don't cover every square inch 
# of the planet!—and so these probabilities reflect our uncertainty about what happened.


# passing in Match_ID m we can access each matches datapoints. In which there is a variable called
# "precipProbability" (number between 0-1) which is returned.

def get_weather(m):

    #build URL
    t = df[df.Match_ID == m]
    lat = str(t.iloc[0]["Latitude"])
    long = str(t.iloc[0]["Longitude"])
    u = int(t.iloc[0]["unixtime"])
    url = ("https://api.darksky.net/forecast/{}/{},{},{}?exclude=currently,minutely,hourly,alerts,flags".format(api_key,lat,long,u))
    
    #get response
    response = requests.get(url)
    obj = response.json()
    
    try:
        weather = (obj.get("daily").get("data")[0]["precipProbability"])
    except:
        weather = "nan"
    return weather

In [20]:
# This line of code is commented out this line because it would access the Darksky API almost 1000 times.
# Instead, the resulting dataframe can be found in "football_weather.csv"
    
# df["rain"] = df["Match_ID"].apply(get_weather)

In [21]:
#exporting the file to .csv format so we wont need to run our get_weather function through the darksky API
# df.to_csv (r'football_weather.csv', index = None, header=True)

# End Phase 1 - Summary

so far in this lab, we have put together all the data we need to answer our questions.  Rather than muddy up and make this notebook too much longer, we have exported our dataframe to a file called "football_weather.csv" and will work with that dataset in part 2.